# 手撕Transformer

> bilibili: AI算法工程师Future

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import math

In [3]:
# test
random_torch = torch.rand(4, 4)
print(random_torch)

tensor([[0.4169, 0.7598, 0.4432, 0.7529],
        [0.2907, 0.1000, 0.9964, 0.4972],
        [0.6539, 0.1523, 0.5657, 0.3042],
        [0.6565, 0.3391, 0.8389, 0.2979]])


In [ ]:
from torch import Tensor
# token embedding
# 将输入的词汇表索引转换为指定维度的Embedding
class TokenEmbedding(nn.Embedding):
    def __init__(self, vocab_size, d_model):
        super(TokenEmbedding, self).__init__(vocab_size, d_model, padding_idx=1)

In [ ]:
class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len, device):
        super(PositionalEmbedding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model, device=device)
        self.encoding.requires_grad = False # 位置编码是预先计算的，不进行梯度更新
        pos = torch.arange(0, max_len, device=device)
        pos = pos.float().unsqueeze(dim=1)
        _2i = torch.arange(0, d_model, step=2, device=device).float()
        self.encoding[:, 0::2] = torch.sin(pos / (10000 ** (_2i / d_model)))
        self.encoding[:, 1::2] = torch.cos(pos / (10000 ** (_2i / d_model)))

    def forward(self, x):
        batch_size, seq_len = x.size()
        return self.encoding[:seq_len, :]

In [ ]:
class TransformerEmbedding(nn.Module):
    def __init__(self, vocal_size, d_model, max_len, drop_prob, device):
        super(TransformerEmbedding, self).__init__()
        self.token_embedding = TokenEmbedding(vocal_size, d_model)
        self.positional_embedding = PositionalEmbedding(d_model, max_len, device)
        self.drop_out = nn.Dropout(drop_prob)

    def forward(self, x):
        token_embed = self.token_embedding(x)
        position_embed = self.positional_embedding(x)
        return self.drop_out(token_embed + position_embed)